# **Dataset bilanciato**

In [ ]:
"""
Montaggio del drive, per poi ricavare i file binari
"""

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""
Questa è l'operazione di caricamento. Per eseguirla è necessario scegliere 
una cartella su drive dove inserire i file binari
"""

import dill 

x_test_12_0 = dill.load(open("/content/drive/MyDrive/Land_12/balanced/x_test_12_0", "rb"))
y_test_12 = dill.load(open("/content/drive/MyDrive/Land_12/balanced/y_test_12", "rb"))

x_train_12_0 = dill.load(open("/content/drive/MyDrive/Land_12/balanced/x_train_12_0", "rb"))
y_train_12 = dill.load(open("/content/drive/MyDrive/Land_12/balanced/y_train_12", "rb"))

# **LSTM e GRU combinati, modello unidirezionale**

In [ ]:
import tensorflow as tf
import dill

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, GRU, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

path = "/content/drive/MyDrive/lstm_gru_uni_best"
callback_check = ModelCheckpoint(path, monitor="val_accuracy", 
                                 save_best_only=True)
callbacks = [callback_check]
initializer_gru = tf.keras.initializers.Constant(value=1)

dict_weights = {0: .50, 1: .50, 2: .50, 3: .25, 4: 1., 5: 1., 6: 0.5}

model = Sequential()

model.add(LSTM(128, input_shape=(x_train_12_0.shape[1:]), return_sequences=True, 
               activation="elu"))
model.add(GRU(128, return_sequences=True, activation="elu"))
model.add(LSTM(128, return_sequences=True, activation="elu"))
model.add(GRU(128, return_sequences=True, activation="elu"))
model.add(LSTM(128, return_sequences=False, activation="elu"))
"""
model.add(GRU(128, return_sequences=True, bias_initializer=initializer_gru, 
              kernel_initializer="glorot_normal"))
model.add(LSTM(128, return_sequences=True, bias_initializer="zeros", 
              kernel_initializer="glorot_normal"))
model.add(GRU(128, return_sequences=True, bias_initializer=initializer_gru,
              kernel_initializer="glorot_normal"))
model.add(LSTM(128, return_sequences=True, bias_initializer="zeros", 
               kernel_initializer="glorot_normal"))
model.add(GRU(128, return_sequences=True, bias_initializer=initializer_gru, 
              kernel_initializer="glorot_normal"))
model.add(LSTM(128, bias_initializer="zeros", 
              kernel_initializer="glorot_normal"))
"""

model.add(Dense(64, activation="elu"))
model.add(Dense(7, activation="softplus"))

opt = tf.optimizers.Adamax(0.0002)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

model.fit(x_train_12_0, y_train_12, batch_size=32, shuffle=True, epochs=800, 
          callbacks=callbacks, validation_data=(x_test_12_0, y_test_12))

# **Tre strati LSTM bidirezionali**

In [ ]:
from tensorflow.keras import backend as b

def modified_elliot(x_tensor):
  x_tensor = (x_tensor / b.sqrt(1 + b.pow(x_tensor, 2))) + 0.5
  return x_tensor

In [ ]:
import tensorflow as tf
import dill

from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint

path = "/content/drive/MyDrive/bid_best"
callback_check = ModelCheckpoint(path, monitor="val_accuracy", 
                                 save_best_only=True)
callbacks = [callback_check]

model = Sequential()

model.add(Bidirectional(LSTM(66, input_shape=(x_train_12_0.shape[1:]), 
                             return_sequences=True, 
                             bias_initializer="glorot_uniform")))
model.add(LSTM(128, return_sequences=True))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(LSTM(128, return_sequences=True))
model.add(Bidirectional(LSTM(128, return_sequences=False)))

model.add(Dense(32, bias_initializer="random_uniform"))
model.add(Dense(7, activation="softplus"))

opt = tf.optimizers.Adadelta(learning_rate=0.001, rho=0.95)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, 
              metrics=["accuracy"])

model.fit(x_train_12_0, y_train_12, batch_size=16, shuffle=True, epochs=400, 
          callbacks=callbacks, validation_data=(x_test_12_0, y_test_12))

# **Rete con LSTM e GRU alternati con Modified Elliot e Orthogonal**

In [ ]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
from tensorflow.keras.callbacks import ModelCheckpoint


initializer_gru = tf.keras.initializers.Constant(value=-1)
initializer_lstm = tf.keras.initializers.Constant(value=1)

path = "/content/drive/MyDrive/lstm_gru_elliot_orthogonal"
callback_check = ModelCheckpoint(path, monitor="val_accuracy", 
                                 save_best_only=True)
callbacks = [callback_check]

model = Sequential()

model.add(LSTM(128, input_shape=(x_train_12_0.shape[1:]), return_sequences=True, 
               bias_initializer="zeros", 
               kernel_initializer="glorot_uniform", 
               recurrent_activation=modified_elliot))

model.add(GRU(128, return_sequences=True, 
              bias_initializer=initializer_gru, 
              kernel_initializer="glorot_uniform", 
              recurrent_activation=modified_elliot))

model.add(LSTM(128, bias_initializer="zeros", 
               kernel_initializer="glorot_uniform", 
               recurrent_activation=modified_elliot))

model.add(Dense(32, bias_initializer="glorot_uniform"))
model.add(Dense(7, activation="softmax"))

opt = tf.optimizers.Adam(0.0002)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

model.fit(x_train_12_0, y_train_12, batch_size=32, shuffle=True, epochs=100, 
          callbacks=callbacks, validation_data=(x_test_12_0, y_test_12))

# **Modello unidirezionale per dataset sbilanciato applicato a dataset bilanciato**

In [ ]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
from tensorflow.keras.callbacks import ModelCheckpoint

path = "/content/drive/MyDrive/uni_best"
callback_check = ModelCheckpoint(path, monitor="val_accuracy", 
                                 save_best_only=True)
callbacks = [callback_check]

model = Sequential()

model.add(LSTM(128, input_shape=(x_train_12_0.shape[1:]), return_sequences=True, 
               kernel_initializer="glorot_normal", dropout=0.5))
model.add(GRU(128, return_sequences=True, dropout=0.5))
model.add(LSTM(128, return_sequences=True, dropout=0.5))
model.add(GRU(128, return_sequences=True, dropout=0.5))
model.add(LSTM(128, return_sequences=True, dropout=0.5))
model.add(GRU(128, return_sequences=True, dropout=0.5))
model.add(LSTM(128, dropout=0.5))

model.add(Dense(32, bias_initializer="glorot_uniform"))
model.add(Dense(7, activation="softmax"))

opt = tf.optimizers.Adamax(0.0002)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

model.fit(x_train_12_0, y_train_12, batch_size=32, shuffle=True, epochs=4000, 
          callbacks=callbacks, validation_data=(x_test_12_0, y_test_12))

# **Modello ConvLSTM2D**

In [ ]:
import dill 

x_test_conv = dill.load(open("/content/drive/MyDrive/Convolutional/without_nose/x_test_0", "rb"))
y_test_conv = dill.load(open("/content/drive/MyDrive/Convolutional/without_nose/y_test", "rb"))

x_train_conv = dill.load(open("/content/drive/MyDrive/Convolutional/without_nose/x_train_0", "rb"))
y_train_conv = dill.load(open("/content/drive/MyDrive/Convolutional/without_nose/y_train", "rb"))

In [ ]:
x_test_conv.shape

(80, 150, 25, 35, 1)

In [ ]:
def create_samples(x, y): 
  samples = []
  for i in range(0, len(x)): 
    samples.append((x[i], y[i]))
  
  return samples

In [ ]:
samples_train = create_samples(x_train_conv, y_train_conv)
samples_test = create_samples(x_test_conv, y_test_conv)

In [ ]:
import numpy as np
import tensorflow as tf

def generator_(samples, batch_size):

  num_samples = len(samples)
  while True:
    for offset in range(0, num_samples, batch_size): 

      batch_samples = samples[offset:offset+batch_size]
      batch_target = samples[offset:offset+batch_size]

      x_train = []
      y_train = []

      for i in range(0, len(batch_samples)): 
        x_train.append(batch_samples[i][0])
        y_train.append(batch_target[i][1])

      x_train = np.array(x_train)
      y_train = np.array(y_train)
      
      yield x_train, y_train

In [ ]:
train_generator = generator_(samples_train, 4)
test_generator = generator_(samples_test, 4)

In [ ]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Dense, Dropout, Flatten

model = Sequential()

model.add(ConvLSTM2D(64, kernel_size=(3, 3), return_sequences=True, 
                     data_format="channels_last", 
                     input_shape=x_train_conv.shape[1:]))
model.add(ConvLSTM2D(64, kernel_size=(3, 3), return_sequences=True, dropout=0.5))
model.add(ConvLSTM2D(64, kernel_size=(3, 3), return_sequences=True, dropout=0.5))
model.add(ConvLSTM2D(64, kernel_size=(3, 3), return_sequences=False))

model.add(Flatten())
model.add(Dense(32))
model.add(Dense(7, activation="softmax"))

opt = tf.keras.optimizers.Adamax(0.0002)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, 
              metrics="accuracy")

model.fit(train_generator, shuffle=True, epochs=50, steps_per_epoch=21, 
          validation_data=(x_test_conv, y_test_conv))

model.evaluate(x_test_conv, y_test_conv)

# **Dataset 8 landmark bilanciato con 7 lingue**

In [ ]:
import dill 

x_test_8_0 = dill.load(open("/content/drive/MyDrive/Land_8/balanced_7_languages/x_test_8_0", "rb"))
y_test_8 = dill.load(open("/content/drive/MyDrive/Land_8/balanced_7_languages/y_test_8", "rb"))

x_train_8_0 = dill.load(open("/content/drive/MyDrive/Land_8/balanced_7_languages/x_train_8_0", "rb"))
y_train_8 = dill.load(open("/content/drive/MyDrive/Land_8/balanced_7_languages/y_train_8", "rb"))

In [ ]:
x_train_8_0.shape

(531, 330, 28)

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers

path = "/content/drive/MyDrive/uni_balanced_best_8_7"
callback_check = ModelCheckpoint(path, monitor="val_accuracy", 
                                 save_best_only=True)
callbacks = [callback_check]

model = Sequential()

model.add(LSTM(128, input_shape=(x_train_8_0.shape[1:]), 
                return_sequences=True, 
                bias_initializer="glorot_uniform"))
model.add(GRU(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True, activity_regularizer=regularizers.l2(1e-5)))
model.add(GRU(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True, activity_regularizer=regularizers.l2(1e-5)))
model.add(GRU(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True, activity_regularizer=regularizers.l2(1e-5)))
model.add(GRU(128, return_sequences=True))
model.add(LSTM(64, return_sequences=True, activity_regularizer=regularizers.l2(1e-5)))
model.add(GRU(64, return_sequences=True))
model.add(LSTM(64, return_sequences=True, activity_regularizer=regularizers.l2(1e-5)))
model.add(GRU(64, return_sequences=True))
model.add(LSTM(64))

model.add(Dense(32, bias_initializer="glorot_uniform"))
model.add(Dense(7, activation="softmax"))

opt = tf.keras.optimizers.Adamax(0.002)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

model.fit(x_train_8_0, y_train_8, batch_size=16, shuffle=True, epochs=400, 
          callbacks=callbacks, validation_data=(x_test_8_0, y_test_8))

# **Modello bidirezionale**

In [ ]:
import tensorflow as tf
import dill

from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint

path = "/content/drive/MyDrive/bid_balanced_best_8_7"
callback_check = ModelCheckpoint(path, monitor="val_accuracy", 
                                 save_best_only=True)
callbacks = [callback_check]

model = Sequential()

model.add(Bidirectional(LSTM(128, input_shape=(x_train_12_0.shape[1:]), 
                             return_sequences=True, 
                             bias_initializer="glorot_uniform")))
model.add(Bidirectional(GRU(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(GRU(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128)))

model.add(Dense(32, bias_initializer="random_uniform"))
model.add(Dense(7, activation="sofmax"))

opt = tf.optimizers.Adagrad(learning_rate=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, 
              metrics=["accuracy"])

model.fit(x_train_8_0, y_train_8, batch_size=32, shuffle=True, epochs=400, 
          callbacks=callbacks, validation_data=(x_test_8_0, y_test_8))

# **Valutazione del modello**

In [ ]:
from tensorflow.keras import models

model_one = models.load_model("/content/drive/MyDrive/bid_balanced_best_8_7")

In [ ]:
model_one.summary()

In [ ]:
label = ["italiano", "inglese", "tedesco", "spagnolo", "olandese", "russo", 
         "giapponese"]

In [ ]:
model_one.evaluate(x_test_8_0, y_test_8)

2/2 [==============================] - 3s 85ms/step - loss: 1.6959 - accuracy: 0.4595


[1.6958529949188232, 0.45945945382118225]

In [ ]:
import numpy as np
predictions = model_one.predict(x_test_8_0)
max_predictions = []

corrette_lingua = {"italiano": 0, "inglese": 0, "tedesco": 0, "spagnolo": 0, 
        "olandese": 0, "russo": 0, "giapponese": 0}
totali_lingua = {"italiano": 0, "inglese": 0, "tedesco": 0, "spagnolo": 0, 
        "olandese": 0, "russo": 0, "giapponese": 0}
for p in predictions: 
  max_predictions.append(max(p))

total = 0
correct = 0
for (i, p) in enumerate(predictions): 
  print("Predizione massima", max_predictions[i], 
        "Lingua: ", label[np.argmax(predictions[i])], 
        "Lingua corretta: ", label[y_test_8[i]] + "\n")
  
  valore = totali_lingua.get(label[np.argmax(predictions[i])])
  valore += 1
  totali_lingua[label[np.argmax(predictions[i])]] = valore

  if (label[y_test_8[i]] == label[np.argmax(predictions[i])]): 
    valore1 = corrette_lingua.get(label[y_test_8[i]])
    valore1 += 1
    corrette_lingua[label[y_test_8[i]]] = valore1
  
print("Corrette per linguae: ", corrette_lingua)
print("Totali per lingua: ", totali_lingua)